In [ ]:
from google.colab import files
uploaded = files.upload()


In [29]:
import csv
import math
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

# === Multi-threaded MapReduce Simulation
# This solution mimics the parallel nature of MapReduce using Python threads.

# Map Phase: Processes a chunk of rows and maps passenger flights
def map_chunk(chunk):
    """
    Map function: Converts a list of rows into key-value pairs (PassengerID, 1)
    """
    return [(row[0], 1) for row in chunk]

def multithreaded_map(filepath, num_threads=4):
    """
    Split the dataset into chunks and process them in parallel using threads
    """
    with open(filepath, mode='r') as file:
        reader = list(csv.reader(file))

    # Divide data into chunks for parallel mapping
    chunk_size = math.ceil(len(reader) / num_threads)
    chunks = [reader[i:i + chunk_size] for i in range(0, len(reader), chunk_size)]

    mapped_results = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = executor.map(map_chunk, chunks)
        for result in futures:
            mapped_results.extend(result)

    return mapped_results

In [28]:
#Reduce Phase: Processes a chunk of mapped pairs
def reduce_chunk(mapped_data):
    """
    Reduce function: Aggregates counts for each PassengerID in a chunk
    """
    local_counts = defaultdict(int)
    for passenger_id, count in mapped_data:
        local_counts[passenger_id] += count
    return local_counts

def merge_counts(partials):
    """
    Merge function: Combines partial counts from different threads
    """
    final_counts = defaultdict(int)
    for partial in partials:
        for pid, count in partial.items():
            final_counts[pid] += count
    return final_counts

def multithreaded_reduce(mapped_data, num_threads=4):
    """
    Run reduce step in parallel across chunks of mapped data
    """
    chunk_size = math.ceil(len(mapped_data) / num_threads)
    chunks = [mapped_data[i:i + chunk_size] for i in range(0, len(mapped_data), chunk_size)]

    partial_results = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = executor.map(reduce_chunk, chunks)
        for result in futures:
            partial_results.append(result)

    return merge_counts(partial_results)

In [27]:
#Execution Starts Here
# Set the path to your uploaded CSV file
filepath = "AComp_Passenger_data_no_error.csv"  # Make sure this is uploaded in Colab

# Map phase using multiple threads
mapped_result = multithreaded_map(filepath, num_threads=4)

# Reduce phase using multiple threads
reduced_result = multithreaded_reduce(mapped_result, num_threads=4)

# Find the maximum number of flights
max_flights = max(reduced_result.values())

# Get all passengers with the maximum number of flights
top_passengers = [pid for pid, count in reduced_result.items() if count == max_flights]

#
#
#Output Results
print(f"Maximum number of flights: {max_flights}")
print(f"Passenger(s) with the most flights: {top_passengers}")

Maximum number of flights: 25
Passenger(s) with the most flights: ['UES9151GS5']


In [30]:
# Save all passenger flight counts to CSV
df_all = pd.DataFrame(reduced_result.items(), columns=["PassengerID", "FlightCount"])
df_all.to_csv("passenger_flight_counts.csv", index=False)

# Save the top passenger(s) separately
df_top = pd.DataFrame(top_passengers, columns=["TopPassengerID"])
df_top["FlightCount"] = max_flights
df_top.to_csv("top_passengers.csv", index=False)

print("✅ Results saved as 'passenger_flight_counts.csv' and 'top_passengers.csv'")

✅ Results saved as 'passenger_flight_counts.csv' and 'top_passengers.csv'
